In [53]:
from pyspark.sql import SparkSession
from pyspark.sql.types import LongType
from pyspark.sql import functions as F
from pyspark.sql.functions import date_format
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.functions import isnan, when, count, col
import pandas as pd
from urllib.request import urlretrieve
from zipfile import ZipFile
# Import libraries 

from pyspark.sql import SparkSession, functions as F
from pyspark.sql.functions import isnan, when, count, col
import pandas as pd
import geopandas as gpd
import folium
from chart_studio import plotly as py
import plotly.graph_objs as go
import statsmodels.api as sm
from statsmodels.formula.api import ols
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.lines import Line2D
import os

/Users/Kasturi/opt/anaconda3/lib/python3.9/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [1]:

#--------------------------------------------------------------------------------------------
# Create a spark session
spark = (
    SparkSession.builder.appName("MAST30034 Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "10g")
    .getOrCreate()
)

22/09/11 20:34:51 WARN Utils: Your hostname, MacBook-Air-3.local resolves to a loopback address: 127.0.0.1; using 192.168.0.66 instead (on interface en0)
22/09/11 20:34:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/11 20:34:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [48]:
exchange_rates = spark.read.csv("../data/exchange_rates.csv")
inflation = spark.read.csv("../data/inflation.csv")
merch_fees = spark.read.csv("../data/merch_fees.csv")
covid = spark.read.csv("../data/covid.csv", header=True)

In [49]:
covid.show(5)

+----------+--------------------+------------+---------+------+-----+---------+---------+----+---+----+--------+
|      date|               state|state_abbrev|confirmed|deaths|tests|positives|recovered|hosp|icu|vent|vaccines|
+----------+--------------------+------------+---------+------+-----+---------+---------+----+---+----+--------+
|2020-01-25|Australian Capita...|         ACT|        0|     0|    0|        0|        0|   0|  0|   0|       0|
|2020-01-25|     New South Wales|         NSW|        3|     0|    0|        3|        0|   0|  0|   0|       0|
|2020-01-25|  Northern Territory|          NT|        0|     0|    0|        0|        0|   0|  0|   0|       0|
|2020-01-25|          Queensland|         QLD|        0|     0|    0|        0|        0|   0|  0|   0|       0|
|2020-01-25|     South Australia|          SA|        0|     0|    0|        0|        0|   0|  0|   0|       0|
+----------+--------------------+------------+---------+------+-----+---------+---------+----+--

In [11]:
data = spark.read.parquet("../data/tables/full_join.parquet")

In [12]:
data.show(5)

+-------------+------------+--------------------+---------+--------------+---------------+--------------------+-----+--------+------+------------------+------------------+--------------------+--------------+-------+-----------+
|merchant_name|merchant_abn|          categories|take_rate|revenue_levels|           name|             address|state|postcode|gender|trans_merchant_abn|      dollar_value|            order_id|order_datetime|user_id|consumer_id|
+-------------+------------+--------------------+---------+--------------+---------------+--------------------+-----+--------+------+------------------+------------------+--------------------+--------------+-------+-----------+
|Felis Limited| 10023283211|furniture, home f...|     0.18|             e|      Todd Long|883 Patty Mountai...|  NSW|    2021|  Male|       10023283211| 212.8573179553598|2430fff1-36c1-483...|    2021-05-02|   3471|        551|
|Felis Limited| 10023283211|furniture, home f...|     0.18|             e|  Adrian Mille

In [13]:
data.createOrReplaceTempView("temp")

a = spark.sql(""" 

SELECT DISTINCT(merchant_name)
FROM temp

""")

a.show()

+--------------------+
|       merchant_name|
+--------------------+
|      Amet Nulla Ltd|
|Volutpat Nulla In...|
|   Pede Praesent Ltd|
|Etiam Bibendum In...|
|Magna Ut Incorpor...|
|Faucibus Leo In C...|
|Varius Et Associates|
|Tellus Imperdiet ...|
|Molestie Tellus F...|
|Class Aptent Taci...|
|Placerat Orci Ass...|
|Vitae Purus Gravi...|
|    Dictum Magna Ltd|
|Sed Leo Cras Inco...|
|    Dui Incorporated|
|        Et Pede Inc.|
| Sagittis Associates|
|Ante Nunc Mauris ...|
|Imperdiet Ullamco...|
|     Ut Molestie LLP|
+--------------------+
only showing top 20 rows



In [56]:
# Code adapted from MAST30034 Tutorial 1
# from the current `tute_1` directory, go back two levels to the `MAST30034` directory
output_relative_dir = '../data/'

# check if it exists as it makedir will raise an error if it does exist
if not os.path.exists(output_relative_dir):
    os.makedirs(output_relative_dir)

# Define the directory names
dirs = ['SA2_boundaries', 'SA2_total_population', 'SA2_income', 'SA2_census']

# now, for each type of data set we will need, we will create the paths
for target_dir in dirs: # taxi_zones should already exist
    if not os.path.exists(output_relative_dir + target_dir):
        os.makedirs(output_relative_dir + target_dir)

In [57]:
SA2_URL_ZIP = "https://www.abs.gov.au/statistics/standards/australian-statistical-geography-standard-asgs-edition-3/jul2021-jun2026/access-and-downloads/digital-boundary-files/SA2_2021_AUST_SHP_GDA2020.zip"

# Define the file names
output_zip = "../data/SA2_boundaries/SA2.zip"

# Download the data
urlretrieve(SA2_URL_ZIP, output_zip)


# Extracting the zip file of the geospatial data

# specifying the zip file name
file_name = "../data/SA2_boundaries/SA2.zip"
  
# opening the zip file in READ mode
with ZipFile(file_name, 'r') as zip:
    # extracting all the files
    zip.extractall(path = "../data/SA2_boundaries/")

In [58]:
sf = gpd.read_file("../data/SA2_boundaries/SA2_2021_AUST_GDA2020.shp")

In [59]:
sf

,SA2_CODE21,SA2_NAME21,CHG_FLAG21,CHG_LBL21,SA3_CODE21,SA3_NAME21,SA4_CODE21,SA4_NAME21,GCC_CODE21,GCC_NAME21,STE_CODE21,STE_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,geometry
0,101021007,Braidwood,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,3418.3525,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((149.58424 -35.44426, 149.58444 -35.4..."
1,101021008,Karabar,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,6.9825,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((149.21899 -35.36738, 149.21800 -35.3..."
2,101021009,Queanbeyan,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,4.7620,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((149.21326 -35.34325, 149.21619 -35.3..."
3,101021010,Queanbeyan - East,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,13.0032,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((149.24034 -35.34781, 149.24024 -35.3..."
4,101021012,Queanbeyan West - Jerrabomberra,0,No change,10102,Queanbeyan,101,Capital Region,1RNSW,Rest of NSW,1,New South Wales,AUS,Australia,13.6748,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((149.19572 -35.36126, 149.19970 -35.3..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2468,901031003,Jervis Bay,0,No change,90103,Jervis Bay,901,Other Territories,9OTER,Other Territories,9,Other Territories,AUS,Australia,67.2296,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"MULTIPOLYGON (((150.69567 -35.18295, 150.69556..."
2469,901041004,Norfolk Island,0,No change,90104,Norfolk Island,901,Other Territories,9OTER,Other Territories,9,Other Territories,AUS,Australia,38.6510,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"MULTIPOLYGON (((167.96325 -29.07212, 167.96326..."
2470,997979799,Migratory - Offshore - Shipping (OT),0,No change,99797,Migratory - Offshore - Shipping (OT),997,Migratory - Offshore - Shipping (OT),99799,Migratory - Offshore - Shipping (OT),9,Other Territories,AUS,Australia,NaN,http://linked.data.gov.au/dataset/asgsed3/SA2/...,None
2471,999999499,No usual address (OT),0,No change,99999,No usual address (OT),999,No usual address (OT),99499,No usual address (OT),9,Other Territories,AUS,Australia,NaN,http://linked.data.gov.au/dataset/asgsed3/SA2/...,None


In [157]:
sf.isna().sum()

SA2_CODE21     0
SA2_NAME21     0
CHG_FLAG21     0
CHG_LBL21      0
SA3_CODE21     0
SA3_NAME21     0
SA4_CODE21     0
SA4_NAME21     0
GCC_CODE21     0
GCC_NAME21     0
STE_CODE21     0
STE_NAME21     0
AUS_CODE21     0
AUS_NAME21     0
AREASQKM21    19
LOCI_URI21     0
geometry      19
dtype: int64

In [158]:
# Checking for null values

nulls = sf.isnull().sum()
flag = 0

for val in nulls:

    # If a null value is found, drop the row
    if val > 0:
        sf.dropna(inplace=True)
        flag = 1
